In [36]:
import boto3

template_file="./cloudfront-authorization-at-edge-template.yaml"
stackname="s3landingzone"

cf = boto3.client('cloudformation', region_name='us-east-1')

In [37]:
with open(template_file) as template_fileobj:
        template_data = template_fileobj.read()
        cf.validate_template(TemplateBody=template_data)


In [38]:
# Helper function to retrieve change set status
def changeSetStatus(change_set_name, client):
    response = client.describe_change_set(
      ChangeSetName=change_set_name,
    )
    return response['Status']


In [ ]:
# Create change set
cs_response = cf.create_change_set(
    StackName=stackname,
    TemplateBody=template_data,
    ChangeSetType="CREATE",
    ChangeSetName=stackname + "-cs",
    Capabilities=['CAPABILITY_IAM']
)

In [55]:
#Remove print statements, here for illustration
cs_response['Id']

'arn:aws:cloudformation:us-east-1:000378343852:changeSet/s3landingzone-cs/90faffc2-beb9-4a07-b050-a30643e307cd'

In [56]:
import time

change_set_name = cs_response['Id']
# Wait until change set status is CREATE_COMPLETE
while True:
  response = changeSetStatus(change_set_name, cf)
  print(str(response))
  time.sleep(10)
  if response == 'CREATE_COMPLETE':
      break

# Execute change set
ex_response = cf.execute_change_set(
  ChangeSetName=change_set_name
)


CREATE_COMPLETE
